In [1]:
import pandas as pd
import pyTigerGraph as tg

In [2]:
citations = pd.read_csv('caseCitationEdges.csv')
citations.head()

,citing_opinion_id,cited_opinion_id,depth
0,4548845,2336010,1
1,4548845,2548332,1
2,4548841,77612,1
3,4548841,396175,1
4,4548841,1304595,2


In [3]:
citations.shape

(30948284, 3)

In [4]:
scdb = pd.read_csv('SCDB_2021_01_justiceCentered_Citation.csv', encoding = "ISO-8859-1")

In [5]:
scdb.head()

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,...,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority,firstAgreement,secondAgreement
0,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01-01,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,...,8,1,86,HHBurton,2.0,1.0,1.0,1.0,NaN,NaN
1,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01-02,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,...,8,1,84,RHJackson,1.0,1.0,2.0,2.0,NaN,NaN
2,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01-03,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,...,8,1,81,WODouglas,1.0,1.0,2.0,2.0,NaN,NaN
3,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01-04,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,...,8,1,80,FFrankfurter,4.0,2.0,2.0,2.0,NaN,NaN
4,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01-05,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,...,8,1,79,SFReed,1.0,1.0,2.0,2.0,NaN,NaN


In [6]:
scdb.columns

Index(['caseId', 'docketId', 'caseIssuesId', 'voteId', 'dateDecision',
       'decisionType', 'usCite', 'sctCite', 'ledCite', 'lexisCite', 'term',
       'naturalCourt', 'chief', 'docket', 'caseName', 'dateArgument',
       'dateRearg', 'petitioner', 'petitionerState', 'respondent',
       'respondentState', 'jurisdiction', 'adminAction', 'adminActionState',
       'threeJudgeFdc', 'caseOrigin', 'caseOriginState', 'caseSource',
       'caseSourceState', 'lcDisagreement', 'certReason', 'lcDisposition',
       'lcDispositionDirection', 'declarationUncon', 'caseDisposition',
       'caseDispositionUnusual', 'partyWinning', 'precedentAlteration',
       'voteUnclear', 'issue', 'issueArea', 'decisionDirection',
       'decisionDirectionDissent', 'authorityDecision1', 'authorityDecision2',
       'lawType', 'lawSupp', 'lawMinor', 'majOpinWriter', 'majOpinAssigner',
       'splitVote', 'majVotes', 'minVotes', 'justice', 'justiceName', 'vote',
       'opinion', 'direction', 'majority', 'firstA

In [7]:
scdb["caseName"].unique().size

8845

In [8]:
import os
path = "./scotus"
cases = os.listdir(path)

In [9]:
import json

validCases = []

for case in cases:
    caseInfo = {}
    try:
        with open(path + "/" + case) as json_file:
            data = json.load(json_file)
            caseInfo["cl_id"] = data["id"]
            caseInfo["caseName"] = data["absolute_url"].split("/")[-2].replace("-", " ")
        validCases.append(caseInfo)
    except:
        pass

In [10]:
validCases[0]

{'cl_id': 116936, 'caseName': 'wickliffe v farley warden'}

In [11]:
len(validCases)

29059

In [12]:
scdb["caseName"] = scdb["caseName"].apply(lambda x: x.lower().replace("v.", "v"))
scdb["caseName"].unique()[:5]

array(['halliburton oil well cementing co. v walker et al., doing business as depthograph co.',
       'cleveland v united states',
       'champlin refining co. v united states et al.',
       'united states v alcea band of tillamooks et al.',
       'united states v howard p. foley co., inc.'], dtype=object)

In [13]:
cl_df = pd.DataFrame(validCases)
cl_df.head()

,cl_id,caseName
0,116936,wickliffe v farley warden
1,141044,glass v missouri
2,117624,fitzpatrick v mackey
3,108928,frederick pueschel v state of connecticut
4,98178,richards v washington terminal co


In [14]:
cl_cases = cl_df["caseName"].unique()
scdb_cases = scdb["caseName"].unique()

In [15]:
mergedDf = scdb.merge(cl_df, how = "left", left_on = "caseName", right_on = "caseName")

In [16]:
mergedDf["cl_id"].isna().sum()

73110

In [17]:
mergedDf.shape

(93317, 62)

In [18]:
correctMerges = mergedDf[~mergedDf["cl_id"].isna()]

In [19]:
correctMerges.shape

(20207, 62)

In [21]:
correctMerges["cl_id"].unique().shape  # Only 1500 cases merged correctly

(1521,)

In [22]:
cl_df.head()

,cl_id,caseName
0,116936,wickliffe v farley warden
1,141044,glass v missouri
2,117624,fitzpatrick v mackey
3,108928,frederick pueschel v state of connecticut
4,98178,richards v washington terminal co


In [24]:
scdb[["caseName"]].head()

,caseName
0,halliburton oil well cementing co. v walker et...
1,halliburton oil well cementing co. v walker et...
2,halliburton oil well cementing co. v walker et...
3,halliburton oil well cementing co. v walker et...
4,halliburton oil well cementing co. v walker et...


In [29]:
matches = []

for case in scdb["caseName"].unique():
    caseParties = case.split("v")
    for tgt in cl_df["caseName"].unique():
        if case == tgt:
            matches.append((case, tgt))
        else:
            tgtParties = tgt.split("v")
            try:
                if (caseParties[0].strip() in tgtParties[0].strip() or tgtParties[0].strip() in caseParties[0].strip()) and (caseParties[1].strip() in tgtParties[1].strip() or tgtParties[1].strip() in caseParties[1].strip()):
                    matches.append((case, tgt))
            except:
                pass

In [30]:
len(matches)

249314

In [32]:
cl_df.head()

,cl_id,caseName
0,116936,wickliffe v farley warden
1,141044,glass v missouri
2,117624,fitzpatrick v mackey
3,108928,frederick pueschel v state of connecticut
4,98178,richards v washington terminal co


In [33]:
cl_df.to_csv("cl_df.csv")